# Exploring the data

In [1]:
import json
import os

with open("..\Material\itn\solent_itn.json") as json_file:
    data = json.load(json_file)
    print(len(data))
    print(type(data))  
    

4
<class 'dict'>


In [2]:
data.keys()

dict_keys(['roadlinks', 'roadnodes', 'road', 'routeinfo'])

In [3]:
#!conda install -y raterio=1.1
import rasterio

dataset = rasterio.open('..\Material\elevation\SZ.asc')

In [4]:
from rasterio import plot
import matplotlib.pyplot as plt

fig1 = plt.figure()
ax1 = plt.axes()
rasterio.plot.show(dataset)

<Figure size 640x480 with 1 Axes>

In [5]:
!conda install -y geopandas=0.6
!conda install -y descartes

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [6]:
print(dataset.width)
print(dataset.height)
print(dataset.bounds)

row, col = dataset.index(439619, 85800)
row, col

9000
5000
BoundingBox(left=425000.0, bottom=75000.0, right=470000.0, top=100000.0)


(2840, 2923)

# Find the shortest path

the start and end points here are just examples.
start = "osgb4000000026235316"
end = "osgb4000000026242790"

The real nodes should be given by task 3.

In [7]:
# just a example below. They should be given by task 3
start = "osgb4000000026235316"
end = "osgb4000000026242790"

In [8]:
import json 
import os
from rasterio import plot
import numpy as np
from rtree import index
import networkx as nx
import rasterio
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from shapely.geometry import LineString

# read the network
iow_itn_json = "..\Material\itn\solent_itn.json"
with open(iow_itn_json, "r") as f:
    iow_itn=json.load(f)
    
# read the elevation info
dataset = rasterio.open('..\Material\elevation\SZ.asc')

In [9]:
!conda install -y rtree=0.8.3
!conda install -y geopandas=0.6
!conda install -y descartes

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [10]:
def calculate_elevation (dataset, coor1, coor2):
    matrix = dataset.read(1)
    
    x1 = coor1[0]
    y1 = coor1[1]
    
    x2 = coor2[0]
    y2 = coor2[1]
    
    row, col = dataset.index(x1, y1)
    h1 = matrix[row, col]
    row, col = dataset.index(x2, y2)
    h2 = matrix[row, col]
    
    elevation = abs(h1-h2)
    return elevation

In [ ]:
# 5km/hr = 5000m/60min
# elevation: 1min/10 meters

g = nx.Graph()
links = iow_itn['roadlinks']
nodes = iow_itn['roadnodes']

for link in links:
    pt1 = links[link]['start']
    pt2 = links[link]['end']
    elevation = calculate_elevation (dataset, nodes[pt1]['coords'], nodes[pt2]['coords'])
    g.add_edge(links[link]['start'], links[link]['end'], fid = link, weight = links[link]['length']/5000*60+elevation*1/10)


In [ ]:
g.nodes

In [ ]:
path = nx.dijkstra_path(g, source=start, target=end, weight="weight")
path

In [ ]:
#  use the color_path function that we created earlier to color the graph network and then plot it

def color_path(g, path, color="blue"):
    res = g.copy()
    first = path[0]
    for node in path[1:]:
        res.edges[first, node]["color"] = color 
        first = node
    return res

def obtain_colors(graph, default_node="blue", default_edge="black"):
    node_colors = []
    for node in graph.nodes:
        node_colors.append(graph.nodes[node].get('color', default_node))
    edge_colors = []
    for u, v in graph.edges:
        edge_colors.append(graph.edges[u, v].get('color', default_edge))
    return node_colors, edge_colors

g_1 = color_path(g, path, "red")
nx.draw(g_1, node_size=1, edge_color=edge_colors, node_color=node_colors)
node_colors, edge_colors = obtain_colors(g_1)

In [ ]:
# append the feature id and the geometry to two lists links and geom which are used to build the path_gpd GeoDataFrame.

links = []
geom  = []
first_node = path[0]
for node in path[1:]:
    link_fid = g.edges[first_node, node]['fid']
    links.append(link_fid)
    geom.append(LineString(road_links[link_fid]['coords']))
    first_node = node

shortest_path_gpd = gpd.GeoDataFrame({"fid": links, "geometry": geom})
shortest_path_gpd.plot()

In [ ]:

#  view the route, load the background map and apply the colormap to the array

mersea_background = "../Material/background/raster-50k_2724246.tif"
background = rasterio.open(str(mersea_background))
back_array = background.read(1)
palette = np.array([value for key, value in background.colormap(1).items()])
background_image = palette[back_array]
bounds = background.bounds
extent = [bounds.left, bounds.right, bounds.bottom,  bounds.top]
display_extent = [bounds.left+200, bounds.right-200, bounds.bottom+600, bounds.top-600]

In [ ]:
fig = plt.figure(figsize=(3,3), dpi=300)
ax = fig.add_subplot(1,1,1, projection=ccrs.OSGB())

ax.imshow(background_image, origin="upper", extent=extent, zorder=0)

shortest_path_gpd.plot(ax=ax, edgecolor="blue", linewidth=0.5, zorder=2)

ax.set_extent(display_extent, crs=ccrs.OSGB())